In [1]:
import csv
import pandas
from sklearn.tree import DecisionTreeClassifier as clf
from sklearn import preprocessing
from sklearn import tree
import pydotplus
from IPython.display import Image
from sklearn import metrics
import pickle

In [35]:
arboles = []
n=11
result=mergeDF("adult_train.csv","adult_test.csv")
result_codificado=cod(result) 
sdf=splitDF1(result_codificado)
sdf2=splitDF2(result_codificado)
for i in range(1,n):
 boot=bootstr(sdf)
 boot2=spm(boot,n-1)
 cc2=Selec(boot2,n-1)
 cc3=Selec2(boot2,n-1)
 ldc=ListaDC(boot2,n-1)
 algo=algor()
 clf_train=ARBOL(algo,cc2,cc3)
 arboles.append(clf_train)
 y_pred=SelecTest(algo,sdf2,ldc)
 print(y_pred)
 #print("Balanced accuracy:",metrics.balanced_accuracy_score(sdf2['less_than_50K'], y_pred))

[1 0 0 ... 1 0 0]
[0 1 1 ... 0 0 1]
[0 1 1 ... 1 0 0]
[0 1 1 ... 1 0 0]
[0 0 1 ... 1 0 0]
[0 1 1 ... 1 0 1]
[0 1 0 ... 1 1 1]
[0 1 1 ... 1 0 0]
[0 0 1 ... 1 0 0]
[0 0 1 ... 1 0 0]


In [55]:
result_codificado.iloc[:32559,:]

,39,2174,0,40,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,less_than_50K
0,33,0,0,12,5,9,2,3,0,4,1,38,0
1,21,0,0,39,3,11,0,5,1,4,1,38,0
2,36,0,0,39,3,1,2,5,0,2,1,38,0
3,11,0,0,39,3,9,2,9,5,2,0,4,0
4,20,0,0,39,3,12,2,3,5,4,0,38,0
5,32,0,0,15,3,6,3,7,1,2,0,22,0
6,35,0,0,44,5,11,2,3,0,4,1,38,1
7,14,109,0,49,3,12,4,9,1,4,0,38,1
8,25,81,0,39,3,9,2,3,0,4,1,38,1
9,20,0,0,78,3,15,2,3,0,2,1,38,1


In [2]:
#merge de los dos dataframes en uno
def mergeDF(at1,at2):
 df = pandas.read_csv(at1)
 df2 = pandas.read_csv(at2)
 frames = [df, df2]
 result = pandas.concat(frames)
 return result

In [3]:
#pasar el dataframe por un codificador que pasa los elementos a numerico
def cod(result):
 codificadores = []
 result_codificado = pandas.DataFrame()
 for variable, valores in result.iteritems():
    le = preprocessing.LabelEncoder()
    le.fit(valores)
    codificadores.append(le)
    result_codificado[variable] = le.transform(valores)
 return result_codificado

In [4]:
#Split del dataframe en dos uno del conjunto de entrenamiento y otro para el test
def splitDF1(result_codificado):
 sdf = result_codificado.iloc[:32559,:]
 return sdf

In [5]:
def splitDF2(result_codificado):
 sdf2 = result_codificado.iloc[32560:,:]
 return sdf2

In [6]:
#bootstraping de N rows del dataframe con replazamiento
def bootstr(sdf):
 #boot=(df.sample(df.shape[0],replace=True))
 boot=(sdf.sample(sdf.shape[0],replace=True))
 return boot

In [7]:
#Subspace Method eligiendo 9 columnas + columna del sueldo asegurando que aparezca en el selector random dandole + peso
def spm(boot,n):
 boot2=boot.sample(n, weights=[0.01, 0.01, 0.01, 0.01, 0.01,0.01,0.01,0.01,0.01,0.01,0.01, 0.01,0.89],axis=1)
 return boot2

In [8]:
def Selec(boot2,n):
 cc2 = boot2.iloc[:,1:n]
 return cc2

In [9]:
def Selec2(boot2,n):
 cc3 = boot2['less_than_50K']
 return cc3

In [10]:
def ListaDC(boot2,n):
 ldc = boot2.iloc[:,1:n].columns.values
 return ldc

In [24]:
def algor():
  algo=tree.DecisionTreeClassifier()
  return algo

In [26]:
def ARBOL(algo,cc2,cc3):
 clf_train = algo.fit(cc2,cc3)
 return clf_train

In [30]:
#seleccionar columnas ldc[0],ldc[1],ldc[2],ldc[3],ldc[4],ldc[5],ldc[6],ldc[7],ldc[8]
def SelecTest(algo,sdf2,ldc):
 X_test=sdf2[[ldc[0],ldc[1],ldc[2],ldc[3],ldc[4],ldc[5],ldc[6],ldc[7],ldc[8]]]
 y_pred = algo.predict(X_test)
 return y_pred

In [23]:
print("Accuracy:",metrics.accuracy_score(df2['less_than_50K'], y_pred))

NameError: name 'df2' is not defined

In [109]:
print("Balanced accuracy:",metrics.balanced_accuracy_score(df2['less_than_50K'], y_pred))

Balanced accuracy: 0.6626800426689738
